The toy model
to get frame, trajectory, put on lstm

In [1]:
import torch
import torch.nn as nn
from data_classes import *

In [2]:
torch.__version__

'0.4.1.post2'

In [3]:
e = ETH()
V = Videodata(e)
empty_frame = V.getFrame(1)

In [70]:
traj = V.read_file('/home/yy/ADL4CV/sgan/datasets/hotel/train/biwi_eth_train.txt') # all trajectories
agent_id = 13
traj_agent = traj[np.where(traj[:, 1]==agent_id)]
traj_agent.shape

(18, 4)

In [138]:
(traj[np.where(traj[:, 1]==11)]).shape

(20, 4)

In [141]:
traj_agents.shape

(8, 2, 3)

In [139]:
traj_agent1 = np.concatenate( ((traj[np.where(traj[:, 1]==11)]), np.zeros([7,4])), axis=0)


In [142]:
traj_agent1 = traj[np.where(traj[:, 1]==11)]
traj_agent2 = traj[np.where(traj[:, 1]==12)]
traj_agent3 = traj[np.where(traj[:, 1]==13)]

In [144]:
traj_agents = np.stack((traj_agent1[0:8,2::], 
                       traj_agent2[0:8,2::], 
                       traj_agent3[0:8,2::]), axis=2) # sequence length = 8, num_fehatures = 2 xy, for one agent
traj_agents = np.concatenate((traj_agents, np.zeros([7,2,3])), axis=0)

In [155]:
input_size = 2 # num_input_features x and y
hidden_size = 1 #? num out at a hidden state
num_layers = 1 # number of hidden states
lstm1 = nn.LSTM(input_size, hidden_size, num_layers)

In [156]:
seq_len = 15
batch_size = 3
input1 = torch.from_numpy(traj_agents).view(seq_len, batch_size, -1).float()

In [157]:
h0 = torch.randn(num_layers, batch_size, hidden_size)
c0 = torch.randn(num_layers, batch_size, hidden_size)
[out1, hidden1] = lstm1(input1,(h0,c0) )

In [49]:
lstm2 = nn.LSTMCell(input_size, hidden_size)

In [50]:
hx = torch.randn(batch_size, hidden_size)
cx = torch.randn(batch_size, hidden_size)
output = []
for i in range(seq_len):
    hx, cx = lstm2(input1[i], (hx, cx))
    output.append(hx)

In [89]:
len(output)

8

In [51]:
class LSTM(nn.Module):

    def __init__(self, input_dim, hidden_dim, batch_size, output_dim=1,
                    num_layers=2):
        super(LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers

        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)

        # Define the output layer
        self.linear = nn.Linear(self.hidden_dim, output_dim)

    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input):
        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        lstm_out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        y_pred = self.linear(lstm_out[-1].view(self.batch_size, -1))
        return y_pred.view(-1)

In [52]:
num_epochs=1
X_train = torch.from_numpy(traj_agent[0:seq_len,2::])
model = LSTM(2, 1, batch_size=1, output_dim=1, num_layers=1)

loss_fn = torch.nn.MSELoss(size_average=False)

optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

#####################
# Train model
#####################

hist = np.zeros(num_epochs)

for t in range(num_epochs):
    # Clear stored gradient
    model.zero_grad()
    
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    model.hidden = model.init_hidden()
    
    # Forward pass
    y_pred = model(X_train)

    loss = loss_fn(y_pred, y_train)
    if t % 100 == 0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

/home/yy/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


RuntimeError: Expected object of type torch.FloatTensor but found type torch.DoubleTensor for argument #4 'mat1'